In [2]:
!pip install rdflib pandas scipy numpy

In [3]:

# Import all required libraries (some are missing)
from rdflib import Graph, URIRef
from rdflib.namespace import RDF

In [4]:
import numpy as np 
import pandas as pd 
from scipy.stats import ttest_ind

In [5]:
# first part result
labels=['non-failing heart','failing heart']
data=[(95,50),(103,35),(99,21),(np.nan,15),(np.nan,7),(np.nan,40)]
data_frame=pd.DataFrame.from_records(data,columns=labels)
t_test=ttest_ind(data_frame['non-failing heart'],data_frame['failing heart'],equal_var=False, nan_policy='omit')
t_test.pvalue

5.3187252634905425e-05

In [6]:
# # Run the T-test for the means of two independent samples using the example data

# # Print the p-value

In [72]:
from rdflib import Graph, URIRef, BNode, Literal
from rdflib.namespace import RDF, XSD, RDFS

In [77]:
# Describe the statistical hypothesis test in machine readable form

# First, we initialize an RDF Graph and bind some prefixes
g = Graph()
g.bind('obo', 'http://purl.obolibrary.org/obo/')
g.bind('ex', 'http://example.org/')

# Next, we define some needed vocabulary
obo = dict()
obo['two sample t-test with unequal variance'] = URIRef('http://purl.obolibrary.org/obo/STATO_0000304')
obo['has specified input'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000293')
obo['has specified output'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000299')
obo['p-value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000175')
obo['has value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001938')
obo['scalar value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001931')
obo['has specified numeric value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001937')
obo['iron-responsive element binding'] = URIRef('http://purl.obolibrary.org/obo/GO_0030350')

# Now, populate the graph with statements
n1=URIRef('http://example.org/n1')
# As an example, the following statement types the resource (blank node) 'n1' as 'two sample t-test with unequal variance'
# n1 = BNode()
g.add((n1, RDF.type, obo['two sample t-test with unequal variance']))

# Now, state that the t-test has iron-responsive element binding as a specified input (one statement to complete)
n2 = BNode()
g.add((n1, obo['has specified input'], n2))
g.add((n2, RDF.type, obo['iron-responsive element binding']))
#g.add(...)


# Next, state that the t-test has the p-value computed above as a specified output (two statements to complete)
#g.add(...)
n3 = BNode()
g.add((n1, obo['has specified output'], n3))
g.add((n3, RDF.type, obo['p-value']))
#g.add(...)

# The literal value of the p-value is represented as a scalar value specification with a specified numeric value (three statements to complete)
n4 = BNode()
g.add((n3, obo['has value specification'], n4))
#g.add(...)

g.add((n4, RDF.type, obo['scalar value specification']))
#g.add(...)
# Type the literal as double
g.add((n4, obo['has specified numeric value'], Literal(t_test.pvalue, datatype=XSD.double)))

# Finally, serialize and print the graph in RDF/XML format

print(g.serialize(format="pretty-xml").decode('utf-8'))

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:obo="http://purl.obolibrary.org/obo/"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <obo:STATO_0000304 rdf:about="http://example.org/n1">
    <obo:OBI_0000299>
      <obo:OBI_0000175 rdf:nodeID="N9997aa532862458b81b8bfcceab2ed14">
        <obo:OBI_0001938>
          <obo:OBI_0001931 rdf:nodeID="Ndd1bca6cddce43688a9640bfeea42c04">
            <obo:OBI_0001937 rdf:datatype="http://www.w3.org/2001/XMLSchema#double">5.3187252634905425e-05</obo:OBI_0001937>
          </obo:OBI_0001931>
        </obo:OBI_0001938>
      </obo:OBI_0000175>
    </obo:OBI_0000299>
    <obo:OBI_0000293>
      <obo:GO_0030350 rdf:nodeID="N9faea1299f744b1590e5c8f3ffbfdc79"/>
    </obo:OBI_0000293>
  </obo:STATO_0000304>
</rdf:RDF>



In [34]:

# Process the machine readable statistical hypothesis test by completing the following SPARQL query that returns the p-value

q = """
PREFIX obo: <http://purl.obolibrary.org/obo/>

SELECT ?pvalue WHERE {
  ?r a obo:STATO_0000304 .
  ?r obo:OBI_0000293 [ a obo:GO_0030350 ] .
  ?r obo:OBI_0000299 [
    a obo:OBI_0000175 ; 
       obo:OBI_0001938 [                          
            a obo:OBI_0001931 ;                    
            obo:OBI_0001937 ?pvalue                 
        ] 
  ].

}
"""
for qs in g.query(q):
    print(qs[0])

5.3187252634905425e-05
